In [ ]:
from langchain_community.document_loaders import TextLoader
#from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter

#from langchain.embeddings import openai
#from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

from pathlib import Path
from configparser import ConfigParser

from pathlib import Path
from dotenv import load_dotenv

import os
#load_dotenv()
BASE_DIR = os.getcwd() #Path(__file__).parents[1]
DB_INIT_FILE = BASE_DIR +'/database.ini'


def db_config(filename: Path = DB_INIT_FILE, section: str = 'postgresql'):
    parser = ConfigParser()
    parser.read(filename)
    if parser.has_section(section):
        params = parser.items(section)
        db = {param[0]: param[1] for param in params}
    else:
        raise Exception(f'Section {section} not found in the {filename} file')
    return db



import psycopg2

def create_db_connection():
    params = db_config()
    try:
        conn = psycopg2.connect(**params)
        return conn
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting", error)
    return None


import requests

model_id = "sentence-transformers/all-MiniLM-L6-v2"
api_url = f"https://api-inference.huggingface.co/pipeline/feature-extraction/{model_id}"
hf_token = os.getenv('HF_ACCESS_TOKEN')
headers = {"Authorization": f"Bearer {hf_token}"}

def QueryModelWithTextChunks(texts):
    response = requests.post(api_url, headers=headers, json={"inputs": texts, "options":{"wait_for_model":True}})
    return response.json()

def SaveEmbeddings(embeddingDF, textsList):
    connection = create_db_connection()
    cursor = connection.cursor()
    try:
        for text, embedding in zip(textsList, embeddingDF):
            cursor.execute(
                "INSERT INTO embeddings (embedding, text) VALUES (%s, %s)",
                (embedding, text)
            )
        connection.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error while writing to DB", error)
    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()



In [ ]:


loader = TextLoader("lease-11-1958.txt", encoding="utf-8")
raw_doc_list = loader.load()
#print(len(docs))
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

doc_chunk_list = text_splitter.split_documents(raw_doc_list)
text_chunk_list = [c.page_content for c in doc_chunk_list]

load_dotenv()

model_output_embeddings = QueryModelWithTextChunks(text_chunk_list)

#import pandas as pd
#embeddings = HuggingFaceEmbeddings()


#embeddings = pd.DataFrame(output)
#print(embeddings)

'''
ref: https://github.com/langchain-ai/langchain/issues/2219

from langchain.vectorstores.pgvector import PGVector
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader

import os

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
texts = TextLoader('data/made-up-story.txt').load()
documents = CharacterTextSplitter(chunk_size=500, chunk_overlap=20).split_documents(texts)

CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg2"),
    host=os.environ.get("PGVECTOR_HOST", "localhost"),
    port=int(os.environ.get("PGVECTOR_PORT", "5432")),
    database=os.environ.get("PGVECTOR_DATABASE", "postgres"),
    user=os.environ.get("PGVECTOR_USER", "postgres"),
    password=os.environ.get("PGVECTOR_PASSWORD", "postgres"),
)

db = PGVector.from_documents(
    embedding=embeddings,
    documents=documents,
    collection_name="test",
    connection_string=CONNECTION_STRING,
)
'''

In [7]:
from langchain.vectorstores.pgvector import PGVector
from langchain.embeddings import HuggingFaceEmbeddings

model_id = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_id)
#ADA_TOKEN_COUNT 

params = db_config()
conStr = f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}:{params['port']}/{params['database']}"
db1 = PGVector.from_documents(embedding=embeddings, 
                              documents=doc_chunk_list, 
                              collection_name="embeddings2", 
                              connection_string=conStr)


In [16]:
#similarQ1 = db1.similarity_search_with_score("rent for the ground", k=2)
similarQ1 = db1.similarity_search_with_score("parking")#, k=2)

for s in similarQ1:
    print(s)


(Document(page_content='10. Not the leave or park any motor or other vehicle in the carriageway\nadjoiining the read of the Building so as to cause annoyance or in-\nconvenience to the Lessor or any lessee or occupier of any flat in the\nbuilding.\n\n11. Not to fix a wireless or telefision aerial to the exterior of the\nsaid flat or building but shall be entitled to use such television\naerial as the Lessor may erect on the roof of the said building, the\nposition of such aerial to be at the absolute discretion of the\nLessor.', metadata={'source': 'lease-11-1958.txt'}), 0.6952251377567562)
(Document(page_content='PART II: [RIGHTS INCLUDED IN DEMISE]\n\t\t\t\n1. The right in common with the Lessor and the Lessees or occupiers\nfor the time being of the other Flats in the Building and all others\nhaving the like right to use for purposes of access to and egress from\nthe Flat first the service road and paths coloured yellow on the said \nplan and second the entrance hall and saircase le